In [1]:
# https://blog.finxter.com/how-to-create-an-interactive-web-application-using-jupyter-notebook/
# https://github.com/binder-examples/voila
# https://github.com/voila-dashboards/voila/tree/stable/notebooks
# https://voila-gallery.org/
from pyproj import Transformer, CRS
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'colab'  # "browser" for pycharm, "notebook" for jupyter
#import kaleido  # to render graphs in plotly as images
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from io import StringIO
import base64
import csv
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML # Start writing code here...
# https://github.com/jupyter-widgets/ipywidgets/issues/3445

In [2]:
DATA_URL = 'https://gist.githubusercontent.com/chriddyp/cb5392c35661370d95f300086accea51/raw/8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/indicators.csv'
EXPLANATION = """\
<div class="app-sidebar">
<p><em>Compare different development indicators.</em><p>

<p>Select what indicators to plot in the dropdowns, and use the slider
to sub-select a fraction of years to include in the plot.</p>

<p>Data and idea copied from the <a href="https://dash.plot.ly/getting-started-part-2">
Plotly Dash documentation</a>.</p>

<p>This example demonstrates combining matplotlib with Jupyter widgets. For more interactive plots,
consider using <a href="https://github.com/bloomberg/bqplot">bqplot</a>.
</div>
"""

In [3]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [38]:
class App:
    
    def __init__(self):
        self._df_local = pd.DataFrame(data={'Element name': 'P1', 'Easting/Rechtswert': 0,\
                                'Northing/Hochwert': 0}, index=[0])
        self._df_global = pd.DataFrame(data={'Element name': 'P1', 'North': 0,\
                                'East': 0}, index=[0])
        self._upload = self._create_upload_button()
        self._tbl = widgets.Output()  # show dataframe
        self._plot_container1 = widgets.Output()  # plot local points
        self._plot_container2 = widgets.Output()  # plot global points
        self._sys_drop = self._create_system_dropdown()
        self._download_button = self._create_download_button()
        self._lang_dropdown = self._create_lang_dropdown()
        self._clickable_link = widgets.Output()
        _app_container = widgets.VBox([
            #widgets.HBox([self._x_dropdown, self._y_dropdown]),
            self._upload,
            self._tbl,
            self._plot_container1,
            self._sys_drop,
            self._plot_container2,
            self._download_button,
            self._lang_dropdown,
            self._clickable_link
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h1>Development indicators</h1>'
                    '<h2 class="app-subtitle"><a href="https://github.com/pbugnion/voila-gallery/blob/master/country-indicators/index.ipynb">Link to code</a></h2>'
                ), 
                layout=widgets.Layout(margin='0 0 5em 0')
            ),
            widgets.HBox([
                _app_container, 
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 2em'))
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()
        
    @classmethod

    # get filename from FileUpload dict
    def get_name(file_upload):
        if len(file_upload) == 0:
            return ''
        return next(iter(file_upload.values()))['metadata']['name']
    
    # display dataframe
    def setup():
        self._df_local.head()

    def _create_upload_button(self):
        upload = widgets.FileUpload(accept='.xlsx', multiple=False)
        upload.observe(self._on_change, names=['value'])
        return upload
    
    def _create_system_dropdown(self):
        # https://blog.finxter.com/how-to-create-an-interactive-web-application-using-jupyter-notebook/

        w = widgets.Dropdown(
            options=['GK Zone 2 - epsg:31466', 'GK Zone 3 - epsg:31467',
            'GK Zone 4 - epsg:31468', 'GK Zone 5 - epsg:31469',
            'UTM ZONE 32 - epsg:32632',
            'UTM ZONE 33 - epsg:32633',
            'MGI Austria GK West - EPSG 31254',
            'MGI Austria GK Central - EPSG 31255',
            'MGI Austria GK East - EPSG 31256',
            'MGI Austria GK M28 - EPSG 31257',
            'MGI Austria GK M31 - EPSG 31258',
            'MGI Austria GK M34 - EPSG 31259'],
            value=None,
            description='Coord system:')
        w.observe(self._on_change, names=['value'])
        return(w)

    def _create_download_button(self):
        button = widgets.Button(
                    description='Download',
                    disabled=False,
                    button_style='', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='Download the converted coordinates',
                    icon='download'
                    )
        button.layout.visibility = "hidden"
        button.on_click(self.create_download_link())
        return button
    
    def _create_lang_dropdown(self):
        dropdown = widgets.Dropdown(options=['English (,)', 'German(;)'],
            value=None,
            description='Laptop language:')
        #dropdown.observe(self._on_change, names=['value'])
        dropdown.layout.visibility = "hidden"
        return dropdown     
    
    def show_it(self):
        if self._upload._counter == 0:  #self._upload.value != {}:
            tbl = pd.DataFrame(data={'Element name': 'P1', 'Easting/Rechtswert': 0,\
                                'Northing/Hochwert': 0}, index=[0])
        else:
            uploaded_filename = next(iter(self._upload.value))
            input_file = list(self._upload.value.values())[0]
            content = input_file['content']  # equal to upload.data[0]
            if uploaded_filename.find('.xls') != -1:
                tbl = pd.read_excel(content, header=0, engine='openpyxl')
            elif uploaded_filename.find('.csv') != -1:
                content = StringIO(content.decode('utf-8'))
                sniffer = csv.Sniffer()
                sniffer.preferred = [';', ',']
                dialect = sniffer.sniff(content)
                tbl = pd.read_csv(content, delimiter=dialect.delimiter, header=0)
            elif uploaded_filename.find('.txt') != -1:
                content = StringIO(content.decode('utf-8'))
                tbl = pd.read_csv(content, sep=" ", header=None)
            # prepare dataframe
            tbl = tbl.rename(columns={tbl.columns[0]: 'Element', tbl.columns[1]: 'X', tbl.columns[2]: 'Y'})
            tbl = tbl.iloc[0:,0:3]
            # drop all but the X and Y columns
            tbl = tbl.astype({'X': 'float64', 'Y': 'float64'})
        with self._tbl:
            display(tbl.head(5))
            self._df_local = tbl
        if self._upload._counter != 0:
            print('File ' + uploaded_filename + ' successfully uploaded. Here\'s a preview')
    
    def _create_plot1(self):
        try:
            if len(self._df_local) > 1:
                if str(self._df_local['X'][0]).find('.') > 2: # draw the points in local coordinates
                    fig1 = px.scatter(self._df_local,
                        x=self._df_local.columns[1],
                        y=self._df_local.columns[2],
                        width=800,  # Width of graph
                        height=600,  # Height of graph
                        #color="region",
                        hover_name=self._df_local.columns[0],
                        #animation_frame = self._df_local.columns[0]
                        )
                    fig1.show(renderer="colab")
                    # give the user some advices
                    if len(self._df_local) > 2:
                        print('Choose a GK Zone. To determine if a GK zone is 2,3, 4 or 5,' +
                        'look at the first number in the Easting/Rechtswert column.\nFor Austria, try the other MGI systems iteratively')
                        if str(self._df_local['X'][0]).find('.') == 6:
                            print('The coordinates could be in UTM, please try that system also')
                        else:
                            print('First number in Easting is: ' + str(self._df_local['X'][0])[0] + ', so it should be Zone ' + str(self._df_local['X'][0])[0])
        except:
            print('Check the file')
            
    
    def _create_plot2(self):
        try:
             if len(self._df_global) > 1:
                if str(self._df_global['North'][0]).find('.') <= 2: # draw the points in global coordinates (with map)
                    fig2 = px.scatter_mapbox(
                            self._df_global,  # Our DataFrame
                            lat='North',
                            lon='East',
                            center={"lat": self._df_global['North'].mean(), "lon": self._df_global['East'].mean()},  # where map will be centered
                            width=800,  # Width of map
                            height=600,  # Height of map
                            hover_data=['Element'],  # what to display when hovering mouse over coordinate
                            #color=self._df_global['Koordinaten Art'],
                            title='Map'
                        )
                    fig2.update_layout(title_x=0.5, mapbox={'style': "open-street-map", 'center': {'lat': self._df_global['North'].mean(),
                                                                                       'lon': self._df_global['East'].mean()}, 'zoom': 18},
                                      hovermode='closest'
                                      # annotations=[
                                      #     dict(
                                      #         x=0.1,
                                      #         y=0.1,
                                      #         align="right",
                                      #         valign="top",
                                      #         text='Koordinaten',
                                      #         showarrow=False,
                                      #         xref="paper",
                                      #         yref="paper",
                                      #         xanchor="center",
                                      #         yanchor="top"
                                      #     )
                                      #]
                                      )
                    # adding beautiful street layout to map and setting initial zoom
                    fig2.show(renderer="colab")
                    #https://plotly.com/python/renderers/
        except:
            print('Check the coordinate system')

    def _calculate(self):
        if self._sys_drop.value != None:
            source = self._sys_drop.value[self._sys_drop.value.find('epsg')::]
            proj_target = CRS.from_string("epsg:4326")  # wgs84
            proj_source = CRS.from_string(source)  # source system
            # for austrian system, Y and X are the normal ones.
            # For german, Y=Hochwert is cartesian X, X=Rechtswert is cartesian Y
            if self._sys_drop.value.find('GK') != -1:
                x = self._df_local['Y']; y = self._df_local['X']
            else:
                x = self._df_local['X']; y = self._df_local['Y']
            transformer = Transformer.from_crs(proj_source, proj_target)
            (nx, ny) = transformer.transform(x, y)
            nx = np.array(nx)[np.newaxis]; nx = nx.T; ny = np.array(ny)[np.newaxis]; ny = ny.T
            self._df_global = pd.DataFrame(data=nx, index=self._df_local['Element'], columns=['North']); self._df_global['East'] = ny;
            self._df_global['Element'] = ''; self._df_global['Element'].iloc[0:] = self._df_local['Element']
            self._df_global = self._df_global[['Element','North', 'East']]
            #print(str(self._df_global['North'].iloc[0]) + ',' + str(self._df_global['East'].iloc[0]))
            self._download_button.layout.visibility = "visible"
            self._lang_dropdown.layout.visibility = "visible"
    
    def create_download_link(self, title = "Download converted coordinates", filename = "Converted_elements.csv"):
        if self._upload._counter != 0:  # otherwise it will try to create a link already, but the dropdown is not yet created
            if self._lang_dropdown.value != None:
                csv = self._df_global.to_csv(index=False, sep=delimiter)
                b64 = base64.b64encode(csv.encode())
                payload = b64.decode()
                html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
                html = html.format(payload=payload,title=title,filename=filename)
                self._clickable_link.display(HTML(html))
                return(HTML(html))
            else:
                print('Select the language of your laptop')

    
    def _on_change(self, _):
        self._update_app()

    def _update_app(self):
        self._tbl.clear_output(wait=True)
        with self._tbl:
            self.show_it()
        self._plot_container1.clear_output(wait=True)
        with self._plot_container1:
            self._create_plot1()
            plt.show()
        self._plot_container2.clear_output(wait=True)
        with self._plot_container2:
            self._calculate()
            self._create_plot2()
            plt.show()

In [39]:
app = App()

app.container

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a82d000-c111-4d9b-b2f0-364d0f452677' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

In [ ]:
import ipywidgets as widgets

text = widgets.HTML('Hyperlink below:')
link = widgets.HTML(
    value="<a href=http://www.google.com target='_blank'>Go to Google idiota</a>",
)

display(text)
display(link)